In [81]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine, text
pd.options.display.float_format = '{:.5f}'.format

In [82]:
protocol ="mysql+pymysql://"
user = "root"
password = "1234"
host = "localhost"
port = "3306"
db = "teamproject"
con_str = f"{protocol}{user}:{password}@{host}:{port}/{db}" 
conn = create_engine(con_str)

In [83]:
Sales_region = pd.read_csv('지역별 전기판매량.csv')
Sales_BF = pd.read_csv('지역별_인상전_후_판매금액.csv')
Generation = pd.read_csv('지역별 발전량.csv')
Sales_use = pd.read_csv('2021용도별전기판매량.csv')
Dept = pd.read_excel('한전부채.xlsx')
Statement = pd.read_excel('한전손익계산서.xlsx')
Dept_BF = pd.read_excel('연도별_인상전후_부채.xlsx')

In [84]:
#데이터 가공

In [85]:
Sales_region.columns = ['지역','2018년','2019년','2020년','2021년']

In [86]:
Sales_BF = Sales_BF.reset_index()
Sales_BF.columns = ['구분','지역','산업용판매량','인상후판매금액','인상전판매금액']
Sales_BF = Sales_BF.iloc[:-1,:]

In [87]:
Generation = Generation.reset_index()
Generation.columns = ['구분','지역','2018년','2019년','2020년','2021년']

In [88]:
Sales_use = Sales_use.transpose().reset_index()
Sales_use.columns = ['구분','주거용전기판매량합계','업무용전기판매량합계','산업용전기판매량합계']
Sales_use.iloc[0,0] = 2021

In [89]:
Dept = Dept.transpose().reset_index()
Dept = Dept.iloc[:,1:]
Dept.columns = Dept.iloc[0,:]
Dept = Dept.iloc[2:,[0,1,2,5,6,7,8,9]]
Dept.columns = ['연도','회계기준','총자산','총금융부채','기타부채','총부채','외화부채','환율_USD']

In [90]:
Statement = Statement.transpose().reset_index().reset_index()
Statement.columns = ['구분','연도','매출액','매출원가','매출총이익','판매비와관리비',
                     '영업이익','기타수익','기타비용','기타이익','금융수익','금융원가','종속기업_관계기업_관련이익',
                     '법인세비용차감전순이익','법인세비용','당기순이익']
Statement = Statement.iloc[1:,:]

In [91]:
#테이블 생성

In [92]:
stmt = """
create table Sales_region(지역 VARCHAR(10) NOT NULL PRIMARY KEY,
                         2018년 DOUBLE NOT NULL,
                         2019년 DOUBLE NOT NULL,
                         2020년 DOUBLE NOT NULL,
                         2021년 DOUBLE NOT NULL
                         )
"""
conn.execute(stmt)

In [93]:
stmt = """
create table Sales_amount(구분 INT NOT NULL PRIMARY KEY,
                         지역 VARCHAR(10) NOT NULL,
                         산업용판매량 DOUBLE NOT NULL,
                         인상후판매금액 DOUBLE NOT NULL,
                         인상전판매금액 DOUBLE NOT NULL
                         )
"""
conn.execute(stmt)

In [94]:
stmt = """
create table Generation_region(구분 INT NOT NULL PRIMARY KEY,
                                지역 VARCHAR(10) NOT NULL,
                                2018년 DOUBLE NOT NULL,
                                2019년 DOUBLE NOT NULL,
                                2020년 DOUBLE NOT NULL,
                                2021년 DOUBLE NOT NULL
                                )
"""
conn.execute(stmt)

In [95]:
stmt = """
create table Sales_use(구분 INT NOT NULL PRIMARY KEY,
                        주거용전기판매량합계 DOUBLE NOT NULL,
                        업무용전기판매량합계 DOUBLE NOT NULL,
                        산업용전기판매량합계 DOUBLE NOT NULL
                        )
"""
conn.execute(stmt)

In [96]:
stmt = """
create table Dept(연도 INT NOT NULL PRIMARY KEY,
                        회계기준 VARCHAR(10) NOT NULL,
                        총자산 FLOAT NOT NULL,
                        총금융부채 INT NOT NULL,
                        기타부채 INT NOT NULL,
                        총부채 FLOAT NOT NULL,
                        외화부채 FLOAT NOT NULL,
                        환율_USD FLOAT NOT NULL
                        )
"""
conn.execute(stmt)

In [97]:
stmt = """
create table Statement(구분 INT NOT NULL PRIMARY KEY,
                        연도 INT NOT NULL,
                        매출액 INT NOT NULL,
                        매출원가 INT NOT NULL,
                        매출총이익 INT NOT NULL,
                        판매비와관리비 INT NOT NULL,
                        영업이익 INT NOT NULL,
                        기타수익 INT NOT NULL,
                        기타비용 INT NOT NULL,
                        기타이익 INT NOT NULL,
                        금융수익 INT NOT NULL,
                        금융원가 INT NOT NULL,
                        종속기업_관계기업_관련이익 INT NOT NULL,
                        법인세비용차감전순이익 INT NOT NULL,
                        법인세비용 INT NOT NULL,
                        당기순이익 INT NOT NULL
                        )
"""
conn.execute(stmt)

In [98]:
stmt = """
create table Dept_BF(연도 INT NOT NULL PRIMARY KEY,
                        연차 INT NOT NULL,
                        인상전후판매금액차이 DOUBLE NOT NULL,
                        부채삭감추이 DOUBLE NOT NULL
                        )
"""
conn.execute(stmt)

In [99]:
#외래키 지정

In [100]:
stmt = '''
ALTER TABLE `Sales_amount` ADD CONSTRAINT `FK_Sales_amount` FOREIGN KEY (`지역`) REFERENCES `Sales_region` (`지역`);
'''
conn.execute(stmt)

In [101]:
stmt = '''
ALTER TABLE `Generation_region` ADD CONSTRAINT `FK_Generation_region` FOREIGN KEY (`지역`) REFERENCES `Sales_region` (`지역`);
'''
conn.execute(stmt)

In [102]:
stmt = '''
ALTER TABLE `Statement` ADD CONSTRAINT `FK_Statement` FOREIGN KEY (`연도`) REFERENCES `Dept` (`연도`);
'''
conn.execute(stmt)

In [103]:
#값 DB에 저장

In [104]:
sql = """
insert into sales_region(지역,2018년,2019년,2020년,2021년)
values(
:지역,
:2018년,
:2019년,
:2020년,
:2021년
)"""
for param in Sales_region.iloc[:,:].to_dict('records'):
    conn.execute(text(sql),param)

In [105]:
sql = """
insert into Sales_amount(구분,지역,산업용판매량,인상후판매금액,인상전판매금액)
values(
:구분,
:지역,
:산업용판매량,
:인상후판매금액,
:인상전판매금액
)"""
for param in Sales_BF.iloc[:,:].to_dict('records'):
    conn.execute(text(sql),param)

In [106]:
sql = """
insert into Generation_region(구분,지역,2018년,2019년,2020년,2021년)
values(
:구분,
:지역,
:2018년,
:2019년,
:2020년,
:2021년
)"""
for param in Generation.iloc[:,:].to_dict('records'):
    conn.execute(text(sql),param)

In [107]:
sql = """
insert into Sales_use(구분,주거용전기판매량합계,업무용전기판매량합계,산업용전기판매량합계)
values(
:구분,
:주거용전기판매량합계,
:업무용전기판매량합계,
:산업용전기판매량합계
)"""
for param in Sales_use.iloc[:,:].to_dict('records'):
    conn.execute(text(sql),param)

In [108]:
sql = """
insert into Dept(연도,회계기준,총자산,총금융부채,기타부채,총부채,외화부채,환율_USD)
values(
:연도,
:회계기준,
:총자산,
:총금융부채,
:기타부채,
:총부채,
:외화부채,
:환율_USD
)"""
for param in Dept.iloc[:,:].to_dict('records'):
    conn.execute(text(sql),param)

In [109]:
sql = """
insert into Statement(구분,연도,매출액,매출원가,매출총이익,판매비와관리비,영업이익,기타수익,기타비용,기타이익,금융수익,금융원가,종속기업_관계기업_관련이익,
법인세비용차감전순이익,법인세비용,당기순이익)
values(
:구분,
:연도,
:매출액,
:매출원가,
:매출총이익,
:판매비와관리비,
:영업이익,
:기타수익,
:기타비용,
:기타이익,
:금융수익,
:금융원가,
:종속기업_관계기업_관련이익,
:법인세비용차감전순이익,
:법인세비용,
:당기순이익
)"""
for param in Statement.iloc[:,:].to_dict('records'):
    conn.execute(text(sql),param)

In [110]:
sql = """
insert into Dept_BF(연도,연차,인상전후판매금액차이,부채삭감추이)
values(
:연도,
:연차,
:인상전후판매금액차이,
:부채삭감추이
)"""
for param in Dept_BF.iloc[:,:].to_dict('records'):
    conn.execute(text(sql),param)